# IBOV: preparação + 3 modelos (LogReg, MLP, RandomForest)
Requisitos: pandas, numpy, scikit-learn

In [2]:
# === IBOV: preparação + 3 modelos (LogReg, MLP, RandomForest) ===
# Requisitos: pandas, numpy, scikit-learn
import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

## 1) Leitura do CSV e detecção robusta de colunas

In [3]:
csv_path = "Dados Históricos - Ibovespa.csv"  # ajuste se necessário
df = pd.read_csv(csv_path)
df.columns = [c.strip() for c in df.columns]  # tira espaços

def parse_ptbr_number(s):
    """Converte '123.456,78' -> 123456.78 e trata K/M/B do Investing (em Volume)."""
    if pd.isna(s): 
        return np.nan
    if isinstance(s, (int, float)): 
        return float(s)
    s = str(s).strip()
    factor = 1.0
    if len(s) > 0 and s[-1] in ["K","M","B"]:
        suf = s[-1]; s = s[:-1]
        factor = {"K":1e3, "M":1e6, "B":1e9}[suf]
    s = s.replace(".", "").replace(",", ".")
    try:
        return float(s) * factor
    except:
        return np.nan

tenta reconhecer nomes comuns em pt-BR/inglês p/ cada coluna

In [4]:
col_data = next((c for c in df.columns if "data" in c.lower() or "date" in c.lower()), None)

fechamento: "Último", "Fechamento", "Preço", "Close"

In [5]:
fechamento_alias = ["último","ultimo","fechamento","preço","preco","close"]
col_close = next((c for c in df.columns if any(a in c.lower() for a in fechamento_alias)), None)
col_open  = next((c for c in df.columns if "abertura" in c.lower() or "open" in c.lower()), None)
col_high  = next((c for c in df.columns if "máx" in c.lower() or "max" in c.lower() or "high" in c.lower()), None)
col_low   = next((c for c in df.columns if "mín" in c.lower() or "min" in c.lower() or "low" in c.lower()), None)
col_vol   = next((c for c in df.columns if "vol" in c.lower()), None)

In [6]:
assert col_data is not None, f"Não encontrei coluna de Data. Cabeçalhos: {list(df.columns)}"
assert col_close is not None, f"Não encontrei coluna de Fechamento ('Último'/'Fechamento'/'Preço'/'Close'). Cabeçalhos: {list(df.columns)}"

conversões

In [7]:
df["Date"]  = pd.to_datetime(df[col_data], dayfirst=True, errors="coerce")
df["Close"] = df[col_close].apply(parse_ptbr_number)
if col_open: df["Open"] = df[col_open].apply(parse_ptbr_number)
if col_high: df["High"] = df[col_high].apply(parse_ptbr_number)
if col_low:  df["Low"]  = df[col_low].apply(parse_ptbr_number)
if col_vol:  df["Volume"] = df[col_vol].apply(parse_ptbr_number)

df = df.sort_values("Date").reset_index(drop=True)

## 2) Target (classe) = 1 se o fechamento do dia seguinte > hoje; senão 0

In [8]:
df["Target"] = (df["Close"].shift(-1) > df["Close"]).astype(int)

## 3) Engenharia de features

In [9]:
df["ret_1d"]  = df["Close"].pct_change(1)
df["ret_5d"]  = df["Close"].pct_change(5)
df["ret_10d"] = df["Close"].pct_change(10)

for w in [5, 10, 20]:
    df[f"ma_{w}"] = df["Close"].rolling(w).mean()
    df[f"ma_ratio_{w}"] = df["Close"] / df[f"ma_{w}"]

df["vola_5d"]  = df["ret_1d"].rolling(5).std()
df["vola_10d"] = df["ret_1d"].rolling(10).std()

def compute_rsi(series, window=14):
    delta = series.diff()
    up = np.where(delta > 0, delta, 0.0)
    down = np.where(delta < 0, -delta, 0.0)
    roll_up = pd.Series(up, index=series.index).rolling(window).mean()
    roll_down = pd.Series(down, index=series.index).rolling(window).mean()
    rs = roll_up / roll_down.replace(0, np.nan)
    return 100.0 - (100.0 / (1.0 + rs))

df["rsi_14"] = compute_rsi(df["Close"], 14)

w = 20
ma = df["Close"].rolling(w).mean()
std = df["Close"].rolling(w).std()
df["bb_width"] = ((ma + 2*std) - (ma - 2*std)) / ma

if "Volume" in df.columns:
    df["vol_z"] = (df["Volume"] - df["Volume"].rolling(20).mean()) / df["Volume"].rolling(20).std()

remove NaN iniciais e a última linha (sem target futuro)

In [10]:
df_model = df.dropna().copy()
df_model = df_model.iloc[:-1, :]

## # 4) Split temporal (últimos 30 dias = teste)

In [11]:
test_size = 30 if len(df_model) >= 100 else min(30, max(5, len(df_model)//5))
train = df_model.iloc[:-test_size, :].copy()
test  = df_model.iloc[-test_size:, :].copy()

feature_cols = [
    "ret_1d","ret_5d","ret_10d",
    "ma_ratio_5","ma_ratio_10","ma_ratio_20",
    "vola_5d","vola_10d",
    "rsi_14","bb_width"
]
if "vol_z" in df_model.columns:
    feature_cols.append("vol_z")

X_train = train[feature_cols].values
y_train = train["Target"].values
X_test  = test[feature_cols].values
y_test  = test["Target"].values

In [12]:
def avalia(nome, modelo):
    modelo.fit(X_train, y_train)
    y_pred = modelo.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    cm  = confusion_matrix(y_test, y_pred)
    cr  = classification_report(y_test, y_pred, digits=3, zero_division=0)
    print(f"\n=== {nome} ===")
    print(f"Acurácia (últimos {test_size} dias): {acc:.4f}")
    print("Matriz de confusão:\n", cm)
    print(cr)
    return acc

## 5) Três modelos

(A) Regressão Logística

In [13]:
logreg = make_pipeline(
    StandardScaler(),
    LogisticRegression(max_iter=2000)
)
acc_logreg = avalia("Regressão Logística", logreg)


=== Regressão Logística ===
Acurácia (últimos 30 dias): 0.5667
Matriz de confusão:
 [[11  3]
 [10  6]]
              precision    recall  f1-score   support

           0      0.524     0.786     0.629        14
           1      0.667     0.375     0.480        16

    accuracy                          0.567        30
   macro avg      0.595     0.580     0.554        30
weighted avg      0.600     0.567     0.549        30



(B) MLP (rede neural)

In [14]:
mlp = make_pipeline(
    StandardScaler(),
    MLPClassifier(hidden_layer_sizes=(64, 64), activation="relu",
                  alpha=1e-3, max_iter=2000, random_state=42)
)
acc_mlp = avalia("MLP (Rede Neural)", mlp)


=== MLP (Rede Neural) ===
Acurácia (últimos 30 dias): 0.5333
Matriz de confusão:
 [[8 6]
 [8 8]]
              precision    recall  f1-score   support

           0      0.500     0.571     0.533        14
           1      0.571     0.500     0.533        16

    accuracy                          0.533        30
   macro avg      0.536     0.536     0.533        30
weighted avg      0.538     0.533     0.533        30



(C) Random Forest

In [15]:
rf = RandomForestClassifier(
    n_estimators=600,
    max_depth=None,
    min_samples_leaf=2,
    max_features="sqrt",
    random_state=42
)
acc_rf = avalia("Random Forest", rf)


=== Random Forest ===
Acurácia (últimos 30 dias): 0.4000
Matriz de confusão:
 [[ 8  6]
 [12  4]]
              precision    recall  f1-score   support

           0      0.400     0.571     0.471        14
           1      0.400     0.250     0.308        16

    accuracy                          0.400        30
   macro avg      0.400     0.411     0.389        30
weighted avg      0.400     0.400     0.384        30



# 6) Comparativo final

In [16]:
melhor = max(
    [("LogReg", acc_logreg), ("MLP", acc_mlp), ("RF", acc_rf)],
    key=lambda x: x[1]
)
print(f"\n>> Melhor entre os 3: {melhor[0]} (acc={melhor[1]:.4f})  |  Meta 75% {'OK' if melhor[1] >= 0.75 else 'NÃO ATINGIDA'}")


>> Melhor entre os 3: LogReg (acc=0.5667)  |  Meta 75% NÃO ATINGIDA


# 7) Tabela de previsões

In [23]:
candidatos = []
if 'rf' in globals() and 'acc_rf' in globals():         candidatos.append(('RF', acc_rf, rf))
if 'logreg' in globals() and 'acc_logreg' in globals(): candidatos.append(('LogReg', acc_logreg, logreg))
if 'mlp' in globals() and 'acc_mlp' in globals():       candidatos.append(('MLP', acc_mlp, mlp))

if candidatos:
    ganhador = max(candidatos, key=lambda x: x[1])
    nome, acc, modelo = ganhador
    print(f"Modelo selecionado automaticamente pelo MAIOR ACC: {nome} ({acc:.4f})")
else:
    # fallback: prioridade rf > logreg > mlp (Opção A)
    if 'rf' in globals():       modelo = rf;      nome = 'RF'
    elif 'logreg' in globals(): modelo = logreg;  nome = 'LogReg'
    elif 'mlp' in globals():    modelo = mlp;     nome = 'MLP'
    else:
        raise RuntimeError("Nenhum modelo está em memória. Rode a célula de treino.")
    print(f"Modelo selecionado por disponibilidade: {nome}")


Modelo selecionado automaticamente pelo MAIOR ACC: LogReg (0.5667)


Tabela de previsões dos últimos 30 dias

In [26]:
# --- Tabela do TESTE (30 dias) ---
test_pred  = modelo.predict(X_test)
proba_test = modelo.predict_proba(X_test)[:,1] if hasattr(modelo,'predict_proba') else None

tabela_teste = test[['Date','Close','Target']].copy()
tabela_teste['pred'] = test_pred
tabela_teste['real_symbol'] = tabela_teste['Target'].map({1:'↑',0:'↓'})
tabela_teste['pred_symbol'] = tabela_teste['pred'].map({1:'↑',0:'↓'})
if proba_test is not None:
    tabela_teste['prob_up'] = proba_test

tabela_teste.tail(30)  # exibe os 30 dias de teste


,Date,Close,Target,pred,real_symbol,pred_symbol,prob_up
718,2025-08-25,138.025,0,0,↓,↓,0.464302
719,2025-08-26,137.771,1,0,↑,↓,0.456685
720,2025-08-27,139.206,1,0,↑,↓,0.475325
721,2025-08-28,141.049,1,0,↑,↓,0.494920
722,2025-08-29,141.422,0,0,↓,↓,0.495973
723,2025-09-01,141.283,0,0,↓,↓,0.489224
724,2025-09-02,140.335,0,1,↓,↑,0.526295
725,2025-09-03,139.864,1,1,↑,↑,0.528850
726,2025-09-04,140.993,1,1,↑,↑,0.532501
727,2025-09-05,142.640,0,0,↓,↓,0.481590


Previsão do próximo dia (Target futuro + probabilidade)

In [27]:
# --- Previsão para o PRÓXIMO DIA (D+1) ---
df_feat_full = df.dropna().copy()  # última linha com todas as features
x_future = df_feat_full.iloc[-1][feature_cols].values.reshape(1, -1)

y_pred = modelo.predict(x_future)[0]
proba  = modelo.predict_proba(x_future)[0, 1] if hasattr(modelo, "predict_proba") else None
data_ref = pd.to_datetime(df_feat_full.iloc[-1]['Date']).date()

print(f"Previsão para o DIA SEGUINTE a {data_ref}: {'Alta (↑)' if y_pred==1 else 'Queda/Estabilidade (↓)'}")
if proba is not None:
    print(f"Probabilidade de alta: {proba:.3f}")

Previsão para o DIA SEGUINTE a 2025-10-06: Queda/Estabilidade (↓)
Probabilidade de alta: 0.462
